# Demonstration of distribution reweighting

**hep_ml.reweight** contains methods to reweight distributions. 
Typically we use reweighting of monte-carlo to fight drawbacks of simulation, though there are many applications.

In this example we reweight multidimensional distibutions: `original` and `target`, the aim is to find new weights for original distribution, such that these multidimensional distributions will coincide. 

Here we have a __toy example__ without a real physics meaning.

Pay attention: equality of distibutions for each feature $\neq$ equality of multivariate distributions.

All samples are divided into **training** and **validation** part. Training part is used to fit reweighting rule and test part is used to estimate reweighting quality.

In [ ]:
%matplotlib inline

import numpy
import pandas
import root_numpy
from matplotlib import pyplot as plt

from hep_ml import reweight

### Downloading data

In [ ]:
storage = "https://github.com/arogozhnikov/hep_ml/blob/data/data_to_download/"
!wget -O ../data/MC_distribution.root -nc $storage/MC_distribution.root?raw=true
!wget -O ../data/RD_distribution.root -nc $storage/RD_distribution.root?raw=true

In [ ]:
columns = ["hSPD", "pt_b", "pt_phi", "vchi2_b", "mu_pt_sum"]

original = root_numpy.root2array("../data/MC_distribution.root", branches=columns)
target = root_numpy.root2array("../data/RD_distribution.root", branches=columns)

original = pandas.DataFrame(original)
target = pandas.DataFrame(target)

original_weights = numpy.ones(len(original))

### prepare train and test samples

* train part is used to train reweighting rule
* test part is used to evaluate reweighting rule comparing the following things: 
    * Kolmogorov-Smirnov distances for 1d projections
    * n-dim distibutions using ML (see below).

In [ ]:
from sklearn.model_selection import train_test_split

# divide original samples into training ant test parts
original_train, original_test = train_test_split(original)
# divide target samples into training ant test parts
target_train, target_test = train_test_split(target)

original_weights_train = numpy.ones(len(original_train))
original_weights_test = numpy.ones(len(original_test))

In [ ]:
from hep_ml.metrics_utils import ks_2samp_weighted

hist_settings = {"bins": 100, "density": True, "alpha": 0.7}


def draw_distributions(original, target, new_original_weights):
    plt.figure(figsize=[15, 7])
    for id, column in enumerate(columns, 1):
        xlim = numpy.percentile(numpy.hstack([target[column]]), [0.01, 99.99])
        plt.subplot(2, 3, id)
        plt.hist(original[column], weights=new_original_weights, range=xlim, **hist_settings)
        plt.hist(target[column], range=xlim, **hist_settings)
        plt.title(column)
        print(
            "KS over ",
            column,
            " = ",
            ks_2samp_weighted(
                original[column],
                target[column],
                weights1=new_original_weights,
                weights2=numpy.ones(len(target), dtype=float),
            ),
        )

### Notice:
Setting `density=True` in `hist_settings` above means that the histograms will be drawn normalized below, so that the area under each histogram integrates to 1. This can obscure the fact that the weights produced by `predict_weights` are not normalized; if you want the number of effective events to be the same after reweighting, you must renormalize the weights manually--see the `reweight` documentation.


## Original distributions
KS = Kolmogorov-Smirnov distance

In [ ]:
# pay attention, actually we have very few data
len(original), len(target)

In [ ]:
draw_distributions(original, target, original_weights)

### train part of original distribution

In [ ]:
draw_distributions(original_train, target_train, original_weights_train)

### test part for target distribution

In [ ]:
draw_distributions(original_test, target_test, original_weights_test)

# Bins-based reweighting in n dimensions
Typical way to reweight distributions is based on bins.

In [ ]:
bins_reweighter = reweight.BinsReweighter(n_bins=20, n_neighs=1.0)
bins_reweighter.fit(original_train, target_train)

bins_weights_test = bins_reweighter.predict_weights(original_test)
# validate reweighting rule on the test part comparing 1d projections
draw_distributions(original_test, target_test, bins_weights_test)

# Gradient Boosted Reweighter

This algorithm is inspired by gradient boosting and is able to fight curse of dimensionality.
It uses decision trees and special loss functiion (**ReweightLossFunction**).

**GBReweighter** supports negative weights (to reweight MC to splotted real data).

In [ ]:
reweighter = reweight.GBReweighter(
    n_estimators=50, learning_rate=0.1, max_depth=3, min_samples_leaf=1000, gb_args={"subsample": 0.4}
)
reweighter.fit(original_train, target_train)

gb_weights_test = reweighter.predict_weights(original_test)
# validate reweighting rule on the test part comparing 1d projections
draw_distributions(original_test, target_test, gb_weights_test)

## Comparing some simple expressions:
the most interesting is checking some other variables in multidimensional distributions (those are expressed via original variables).

In [ ]:
def check_ks_of_expression(expression):
    col_original = original_test.eval(expression, engine="python")
    col_target = target_test.eval(expression, engine="python")
    w_target = numpy.ones(len(col_target), dtype="float")
    print(
        "No reweight   KS:",
        ks_2samp_weighted(col_original, col_target, weights1=original_weights_test, weights2=w_target),
    )
    print(
        "Bins reweight KS:", ks_2samp_weighted(col_original, col_target, weights1=bins_weights_test, weights2=w_target)
    )
    print("GB Reweight   KS:", ks_2samp_weighted(col_original, col_target, weights1=gb_weights_test, weights2=w_target))

In [ ]:
check_ks_of_expression("hSPD")

In [ ]:
check_ks_of_expression("hSPD * pt_phi")

In [ ]:
check_ks_of_expression("hSPD * pt_phi * vchi2_b")

In [ ]:
check_ks_of_expression("pt_b * pt_phi / hSPD ")

In [ ]:
check_ks_of_expression("hSPD * pt_b * vchi2_b / pt_phi")

# GB-discrimination
let's check how well a classifier is able to distinguish these distributions. ROC AUC is taken as a measure of quality.

For this puprose we split the data into train and test, then we train a classifier do distinguish these distributions.
If ROC AUC = 0.5 on test, distibutions are identical, if ROC AUC = 1.0, they are ideally separable.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

data = numpy.concatenate([original_test, target_test])
labels = numpy.array([0] * len(original_test) + [1] * len(target_test))

weights = {}
weights["original"] = original_weights_test
weights["bins"] = bins_weights_test
weights["gb_weights"] = gb_weights_test


for name, new_weights in weights.items():
    W = numpy.concatenate([new_weights / new_weights.sum() * len(target_test), [1] * len(target_test)])
    Xtr, Xts, Ytr, Yts, Wtr, Wts = train_test_split(data, labels, W, random_state=42, train_size=0.51)
    clf = GradientBoostingClassifier(subsample=0.3, n_estimators=50).fit(Xtr, Ytr, sample_weight=Wtr)

    print(name, roc_auc_score(Yts, clf.predict_proba(Xts)[:, 1], sample_weight=Wts))

# Folding reweighter

With `FoldingReweighter` one can simpler do cross-validation and in the end obtain unbiased weights for the whole original sample

In [ ]:
# define base reweighter
reweighter_base = reweight.GBReweighter(
    n_estimators=50, learning_rate=0.1, max_depth=3, min_samples_leaf=1000, gb_args={"subsample": 0.4}
)
reweighter = reweight.FoldingReweighter(reweighter_base, n_folds=2)
# it is not needed divide data into train/test parts; rewighter can be train on the whole samples
reweighter.fit(original, target)

# predict method provides unbiased weights prediction for the whole sample
# folding reweighter contains two reweighters, each is trained on one half of samples
# during predictions each reweighter predicts another half of samples not used in training
folding_weights = reweighter.predict_weights(original)

draw_distributions(original, target, folding_weights)

### GB discrimination for reweighting rule

In [ ]:
data = numpy.concatenate([original, target])
labels = numpy.array([0] * len(original) + [1] * len(target))

weights = {}
weights["original"] = original_weights
weights["2-folding"] = folding_weights


for name, new_weights in weights.items():
    W = numpy.concatenate([new_weights / new_weights.sum() * len(target), [1] * len(target)])
    Xtr, Xts, Ytr, Yts, Wtr, Wts = train_test_split(data, labels, W, random_state=42, train_size=0.51)
    clf = GradientBoostingClassifier(subsample=0.3, n_estimators=30).fit(Xtr, Ytr, sample_weight=Wtr)

    print(name, roc_auc_score(Yts, clf.predict_proba(Xts)[:, 1], sample_weight=Wts))